In [236]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import csv
import os
import requests, json

url = "https://localhost:9200/books/_search"
requestHeaders = {'content-type': 'application/json'}

# create new custom metric function for score, rating,similarity 

def metric1(x):
    return x[0]*0.5 + x[1]*0.25 + x[2]*0.25



In [237]:
es = Elasticsearch(['https://localhost:9200/'], ssl_assert_fingerprint="5d62e0cf4920a91659a61a9ad0ac417c02161538a931ec881f16cc842ce88b3d",basic_auth=("elastic", "HYyElLskPkbcjmpiIskE"))



In [238]:
books= pd.DataFrame()

    #-----Elasticsearch metric-----
book = input("What is the book you want to search for?\n")
query = {
            "match": {
                "book_title":book
            }
        }

results = es.search(index='books',query=query)
books = pd.json_normalize(results['hits']['hits'])
books

#Keep stats
scores = books["_score"]
isbn = books["_source.isbn"]

isbn

0    0002005018
1    1931561168
2    0312135084
3    0684844494
4    0819310573
5    0815410255
6    1555466419
7    0679823115
8    0451178734
9    1932270116
Name: _source.isbn, dtype: object

In [239]:

#books = books.set_index("_source.isbn")



In [240]:
books

,_index,_id,_score,_source.isbn,_source.book_title,_source.book_author,_source.year_of_publication,_source.publisher,_source.summary,_source.category
0,books,S_AIRIUBvwM5wp6acRd0,12.645481,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",['actresses']
1,books,WfAIRIUBvwM5wp6adTpY,11.621932,1931561168,Clara Mondschein's Melancholia,Anne Raeff,2002,MacAdam/Cage Publishing,At the heart of this ambitious novel is the qu...,['fiction']
2,books,hPAIRIUBvwM5wp6adkn0,11.621932,0312135084,Henry and Clara,Thomas Mallon,1995,Picador USA,An engaged couple&#39;s lives are forever chan...,['fiction']
3,books,nPEIRIUBvwM5wp6aq9qV,11.621932,0684844494,Clara : A Novel,Janice Galloway,2003,Simon & Schuster,In its luminous integrity the novel brings Cla...,['fiction']
4,books,OPEIRIUBvwM5wp6ajgM5,10.751672,0819310573,Clara Joins the Circus,Michael Pellowski,1982,Granite Impex Ltd,"Clara Cow, in search of excitement, tries to j...",['juvenile fiction']
5,books,EPAIRIUBvwM5wp6aidyo,10.751672,0815410255,Clara Bow: Runnin' Wild,David Stenn,2000,Cooper Square Press,Silent screen goddess Clara Bow was the embodi...,['performing arts']
6,books,4_AIRIUBvwM5wp6aiuBY,10.002664,1555466419,Clara Barton (Women of Achievement),Leni Hamilton,1987,Chelsea House Publications,Follows the life of the nurse who served on th...,['social science']
7,books,LvEIRIUBvwM5wp6aqMMy,9.351215,0679823115,Sweet Clara and the Freedom Quilt,Deborah Hopkinson,1993,Alfred A. Knopf,A young slave stitches a quilt with a map patt...,['juvenile fiction']
8,books,vvEIRIUBvwM5wp6aqtSo,9.351215,0451178734,Dancing With Clara (Signet Regency Romance),Mary Balogh,1994,Signet Book,Miss Clara Danford decides that she must be ca...,['fiction']
9,books,t_EIRIUBvwM5wp6alkmt,8.779435,1932270116,Driven to Kill: The Clara Harris Story,Cliff Linedecker,2003,AMI Books,The agonizing true tale of a woman betrayed.,['true crime']


In [241]:
#Search user ratings:
user_id = int(input("give your uid?\n"))

query = {
    "match":{
        "uid":user_id
    }

}

resp = es.search(index="ratings" , query=query)

In [242]:
resp=pd.json_normalize(resp['hits']['hits'])
resp

,_index,_id,_score,_source.uid,_source.isbn,_source.rating
0,ratings,b_IIRIUBvwM5wp6awiWW,1.0,8,0002005018,5
1,ratings,ffIIRIUBvwM5wp6awiWW,1.0,8,0060973129,0
2,ratings,gPIIRIUBvwM5wp6awiWW,1.0,8,0374157065,0
3,ratings,i_IIRIUBvwM5wp6awiWW,1.0,8,0393045218,0
4,ratings,jPIIRIUBvwM5wp6awiWW,1.0,8,0399135782,0
5,ratings,rfIIRIUBvwM5wp6awiWW,1.0,8,0425176428,0
6,ratings,svIIRIUBvwM5wp6awiWW,1.0,8,0671870432,0
7,ratings,tfIIRIUBvwM5wp6awiWW,1.0,8,0679425608,0
8,ratings,tvIIRIUBvwM5wp6awiWW,1.0,8,074322678X,5
9,ratings,t_IIRIUBvwM5wp6awiWW,1.0,8,0771074670,0


In [243]:
search_user_rating = [0]*len(scores)
for i in range(len(isbn)):
    for j in range(len(resp['_source.isbn'])):
        if isbn[i] ==resp['_source.isbn'][j]:
            search_user_rating[i] = resp['_source.isbn'][j]



In [244]:
#check for non empty list
if [x for x in search_user_rating if x]:
    print("ok")
else:
    print("den uparxei antistixoia metaksi book and user rating")


ok


In [245]:
#B25 SIMILARITY METRIC( DEFAULT ELASTIC METRIC)
print(books[["_score","_source.book_title"]])

      _score                           _source.book_title
0  12.645481                                 Clara Callan
1  11.621932               Clara Mondschein's Melancholia
2  11.621932                              Henry and Clara
3  11.621932                              Clara : A Novel
4  10.751672                       Clara Joins the Circus
5  10.751672                      Clara Bow: Runnin' Wild
6  10.002664          Clara Barton (Women of Achievement)
7   9.351215            Sweet Clara and the Freedom Quilt
8   9.351215  Dancing With Clara (Signet Regency Romance)
9   8.779435       Driven to Kill: The Clara Harris Story


In [246]:

#for i in range(len(books)):
    #books['_score'][i] = metric([scores[i],float(search_user_rating[i])])

In [247]:
#new_books = books.sort_values(by=["_score"],ascending=False)

In [248]:
#new_books=new_books[['_score',"_source.book_title"]]

In [249]:
#new_books

In [250]:
books

,_index,_id,_score,_source.isbn,_source.book_title,_source.book_author,_source.year_of_publication,_source.publisher,_source.summary,_source.category
0,books,S_AIRIUBvwM5wp6acRd0,12.645481,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",['actresses']
1,books,WfAIRIUBvwM5wp6adTpY,11.621932,1931561168,Clara Mondschein's Melancholia,Anne Raeff,2002,MacAdam/Cage Publishing,At the heart of this ambitious novel is the qu...,['fiction']
2,books,hPAIRIUBvwM5wp6adkn0,11.621932,0312135084,Henry and Clara,Thomas Mallon,1995,Picador USA,An engaged couple&#39;s lives are forever chan...,['fiction']
3,books,nPEIRIUBvwM5wp6aq9qV,11.621932,0684844494,Clara : A Novel,Janice Galloway,2003,Simon & Schuster,In its luminous integrity the novel brings Cla...,['fiction']
4,books,OPEIRIUBvwM5wp6ajgM5,10.751672,0819310573,Clara Joins the Circus,Michael Pellowski,1982,Granite Impex Ltd,"Clara Cow, in search of excitement, tries to j...",['juvenile fiction']
5,books,EPAIRIUBvwM5wp6aidyo,10.751672,0815410255,Clara Bow: Runnin' Wild,David Stenn,2000,Cooper Square Press,Silent screen goddess Clara Bow was the embodi...,['performing arts']
6,books,4_AIRIUBvwM5wp6aiuBY,10.002664,1555466419,Clara Barton (Women of Achievement),Leni Hamilton,1987,Chelsea House Publications,Follows the life of the nurse who served on th...,['social science']
7,books,LvEIRIUBvwM5wp6aqMMy,9.351215,0679823115,Sweet Clara and the Freedom Quilt,Deborah Hopkinson,1993,Alfred A. Knopf,A young slave stitches a quilt with a map patt...,['juvenile fiction']
8,books,vvEIRIUBvwM5wp6aqtSo,9.351215,0451178734,Dancing With Clara (Signet Regency Romance),Mary Balogh,1994,Signet Book,Miss Clara Danford decides that she must be ca...,['fiction']
9,books,t_EIRIUBvwM5wp6alkmt,8.779435,1932270116,Driven to Kill: The Clara Harris Story,Cliff Linedecker,2003,AMI Books,The agonizing true tale of a woman betrayed.,['true crime']


In [251]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

bookss=books[['_source.summary','_source.book_title']]

In [252]:
def comb_features(row):
    return row['_source.summary'] + " " + row["_source.book_title"]

bookss['new_feature'] = bookss.apply(comb_features,axis=1)
bookss

/tmp/ipykernel_6163/4006454028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bookss['new_feature'] = bookss.apply(comb_features,axis=1)


,_source.summary,_source.book_title,new_feature
0,"In a small town in Canada, Clara Callan reluct...",Clara Callan,"In a small town in Canada, Clara Callan reluct..."
1,At the heart of this ambitious novel is the qu...,Clara Mondschein's Melancholia,At the heart of this ambitious novel is the qu...
2,An engaged couple&#39;s lives are forever chan...,Henry and Clara,An engaged couple&#39;s lives are forever chan...
3,In its luminous integrity the novel brings Cla...,Clara : A Novel,In its luminous integrity the novel brings Cla...
4,"Clara Cow, in search of excitement, tries to j...",Clara Joins the Circus,"Clara Cow, in search of excitement, tries to j..."
5,Silent screen goddess Clara Bow was the embodi...,Clara Bow: Runnin' Wild,Silent screen goddess Clara Bow was the embodi...
6,Follows the life of the nurse who served on th...,Clara Barton (Women of Achievement),Follows the life of the nurse who served on th...
7,A young slave stitches a quilt with a map patt...,Sweet Clara and the Freedom Quilt,A young slave stitches a quilt with a map patt...
8,Miss Clara Danford decides that she must be ca...,Dancing With Clara (Signet Regency Romance),Miss Clara Danford decides that she must be ca...
9,The agonizing true tale of a woman betrayed.,Driven to Kill: The Clara Harris Story,The agonizing true tale of a woman betrayed. D...


In [253]:
cv=CountVectorizer()
final_matrix = cv.fit_transform(bookss["new_feature"])
final_matrix = final_matrix.toarray()
final_matrix

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [254]:
cosine_similarity = cosine_similarity(final_matrix)
cosine_similarity

array([[1.        , 0.1440976 , 0.07862136, 0.28867513, 0.28074496,
        0.16205093, 0.15575224, 0.18184824, 0.15130688, 0.14173668],
       [0.1440976 , 1.        , 0.21147747, 0.33277916, 0.22654619,
        0.2353796 , 0.35610392, 0.14674182, 0.12209657, 0.2668725 ],
       [0.07862136, 0.21147747, 1.        , 0.15130688, 0.15450786,
        0.2853909 , 0.20953331, 0.16012815, 0.15544046, 0.17334381],
       [0.28867513, 0.33277916, 0.15130688, 1.        , 0.3646984 ,
        0.26196842, 0.32971986, 0.25197632, 0.20965697, 0.38188131],
       [0.28074496, 0.22654619, 0.15450786, 0.3646984 , 1.        ,
        0.28661839, 0.38260825, 0.28589668, 0.20814536, 0.37139068],
       [0.16205093, 0.2353796 , 0.2853909 , 0.26196842, 0.28661839,
        1.        , 0.37691428, 0.26403934, 0.24715577, 0.30012252],
       [0.15575224, 0.35610392, 0.20953331, 0.32971986, 0.38260825,
        0.37691428, 1.        , 0.31722063, 0.17596235, 0.48076197],
       [0.18184824, 0.14674182, 0.1601281

In [255]:
df_new =bookss.pivot_table(index='_source.book_title' ,columns = '_source.summary')

/tmp/ipykernel_6163/3841840223.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  df_new =bookss.pivot_table(index='_source.book_title' ,columns = '_source.summary')


In [256]:
x=list()
for i in range(len(books)):
    x.append(i)

In [257]:
x=pd.DataFrame(x)
con = pd.concat([x,books],axis=1, ignore_index=False)
index_val=list()

for i in range(len(books)):
    in_val=con[con['_source.book_title']== books['_source.book_title'][i]].index.values
    index_val.append(in_val)
index_val

[array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([4]),
 array([5]),
 array([6]),
 array([7]),
 array([8]),
 array([9])]

In [258]:
import numpy as np
for i in range(len(index_val)):
        similar_books =list(enumerate(cosine_similarity[index_val[i]]))
        books["_score"][i] = metric1([scores[i], float(search_user_rating[i]), float(similar_books[0][1][i])])

books

/tmp/ipykernel_6163/1120988229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books["_score"][i] = metric1([scores[i], float(search_user_rating[i]), float(similar_books[0][1][i])])


,_index,_id,_score,_source.isbn,_source.book_title,_source.book_author,_source.year_of_publication,_source.publisher,_source.summary,_source.category
0,books,S_AIRIUBvwM5wp6acRd0,501261.072740,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",['actresses']
1,books,WfAIRIUBvwM5wp6adTpY,6.060966,1931561168,Clara Mondschein's Melancholia,Anne Raeff,2002,MacAdam/Cage Publishing,At the heart of this ambitious novel is the qu...,['fiction']
2,books,hPAIRIUBvwM5wp6adkn0,6.060966,0312135084,Henry and Clara,Thomas Mallon,1995,Picador USA,An engaged couple&#39;s lives are forever chan...,['fiction']
3,books,nPEIRIUBvwM5wp6aq9qV,6.060966,0684844494,Clara : A Novel,Janice Galloway,2003,Simon & Schuster,In its luminous integrity the novel brings Cla...,['fiction']
4,books,OPEIRIUBvwM5wp6ajgM5,5.625836,0819310573,Clara Joins the Circus,Michael Pellowski,1982,Granite Impex Ltd,"Clara Cow, in search of excitement, tries to j...",['juvenile fiction']
5,books,EPAIRIUBvwM5wp6aidyo,5.625836,0815410255,Clara Bow: Runnin' Wild,David Stenn,2000,Cooper Square Press,Silent screen goddess Clara Bow was the embodi...,['performing arts']
6,books,4_AIRIUBvwM5wp6aiuBY,5.251332,1555466419,Clara Barton (Women of Achievement),Leni Hamilton,1987,Chelsea House Publications,Follows the life of the nurse who served on th...,['social science']
7,books,LvEIRIUBvwM5wp6aqMMy,4.925607,0679823115,Sweet Clara and the Freedom Quilt,Deborah Hopkinson,1993,Alfred A. Knopf,A young slave stitches a quilt with a map patt...,['juvenile fiction']
8,books,vvEIRIUBvwM5wp6aqtSo,4.925607,0451178734,Dancing With Clara (Signet Regency Romance),Mary Balogh,1994,Signet Book,Miss Clara Danford decides that she must be ca...,['fiction']
9,books,t_EIRIUBvwM5wp6alkmt,4.639717,1932270116,Driven to Kill: The Clara Harris Story,Cliff Linedecker,2003,AMI Books,The agonizing true tale of a woman betrayed.,['true crime']


In [259]:
books = books.sort_values(by=['_score'], ascending=False)


In [260]:
print(books[["_score", "_source.book_title"]])

          _score                           _source.book_title
0  501261.072740                                 Clara Callan
1       6.060966               Clara Mondschein's Melancholia
2       6.060966                              Henry and Clara
3       6.060966                              Clara : A Novel
4       5.625836                       Clara Joins the Circus
5       5.625836                      Clara Bow: Runnin' Wild
6       5.251332          Clara Barton (Women of Achievement)
7       4.925607            Sweet Clara and the Freedom Quilt
8       4.925607  Dancing With Clara (Signet Regency Romance)
9       4.639717       Driven to Kill: The Clara Harris Story
